In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
import os
import numpy as np
import cv2
from tensorflow import keras
import matplotlib.pyplot as plt


# Path to the folders containing images and masks
image_folder = "/content/gdrive/MyDrive/ORGINAL_img"
mask_folder = "/content/gdrive/MyDrive/ORIGINAL_mask"

# Get the list of image and mask filenames
image_filenames = sorted(os.listdir(image_folder))
mask_filenames = sorted(os.listdir(mask_folder))

# Data normalization function
def normalize_data(data):
    return data / 255.0

# Initialize empty lists to store normalized images and masks
normalized_images = []
normalized_masks = []

# Iterate over the image and mask filenames
for i in range(len(image_filenames)):
    # Load image and mask
    image_filename = os.path.join(image_folder, image_filenames[i])
    mask_filename = os.path.join(mask_folder, mask_filenames[i])
    image = cv2.imread(image_filename)
    mask = cv2.imread(mask_filename, cv2.IMREAD_GRAYSCALE)
    new_size = (384, 96)
    image_resized = cv2.resize(image, new_size)
    mask_resized = cv2.resize(mask, new_size)
    # Normalize image and mask
    normalized_image = normalize_data(image_resized)
    normalized_mask = normalize_data(mask_resized)
    # Binarize the mask (optional)
    normalized_mask = np.where(normalized_mask > 0.5, 1, 0)

    # Append normalized images and masks to the respective lists
    normalized_images.append(normalized_image)
    normalized_masks.append(normalized_mask)

# Convert the image and mask lists to numpy arrays
normalized_images = np.array(normalized_images)
normalized_masks = np.array(normalized_masks)

# Split the data into training and validation sets (adjust the split ratio as desired)
split_ratio = 0.8
split_index = int(len(normalized_images) * split_ratio)
train_images = normalized_images[:split_index]
train_masks = normalized_masks[:split_index]
val_images = normalized_images[split_index:]
val_masks = normalized_masks[split_index:]

# Convert validation masks to labels
val_labels = val_masks[..., np.newaxis]

# U-Net model for binarization
def unet():
    inputs = keras.layers.Input(shape=(96, 384, 3))

    # Encoder
    conv1 = keras.layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = keras.layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = keras.layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = keras.layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = keras.layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = keras.layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = keras.layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = keras.layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = keras.layers.Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = keras.layers.Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Decoder
    up6 = keras.layers.Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(conv5)
    concat6 = keras.layers.concatenate([up6, conv4], axis=3)
    conv6 = keras.layers.Conv2D(512, 3, activation='relu', padding='same')(concat6)
    conv6 = keras.layers.Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = keras.layers.Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv6)
    concat7 = keras.layers.concatenate([up7, conv3], axis=3)
    conv7 = keras.layers.Conv2D(256, 3, activation='relu', padding='same')(concat7)
    conv7 = keras.layers.Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = keras.layers.Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv7)
    concat8 = keras.layers.concatenate([up8, conv2], axis=3)
    conv8 = keras.layers.Conv2D(128, 3, activation='relu', padding='same')(concat8)
    conv8 = keras.layers.Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = keras.layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same') (conv8)
    concat9 = keras.layers.concatenate([up9, conv1], axis=3)
    conv9 = keras.layers.Conv2D(64, 3, activation='relu', padding='same')(concat9)
    conv9 = keras.layers.Conv2D(64, 3, activation='relu', padding='same')(conv9)

    output = keras.layers.Conv2D(1, 1, activation='sigmoid')(conv9)

    model = keras.models.Model(inputs=inputs, outputs=output)
    return model

# Create the U-Net model
model = unet()

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model and collect the history
history = model.fit(train_images, train_masks, validation_data=(val_images, val_masks), epochs=20, batch_size=8)

# Get the training metrics
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Plot the training loss and validation loss
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot the training accuracy and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

# Display the plots
plt.tight_layout()
plt.show()


In [ ]:
model.save("U-NET_whole_image.h5")

In [ ]:
from tensorflow import keras
model = keras.models.load_model("/content/gdrive/MyDrive/U-NET_whole_image.h5")

In [ ]:



import os
import numpy as np
import cv2
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score, roc_curve, auc
from skimage.metrics import structural_similarity

# Path to the folders containing images and masks
image_folder = "/content/gdrive/MyDrive/TEST_original"
mask_folder = "/content/gdrive/MyDrive/TEST_mask"

# Get the list of image and mask filenames
image_filenames = sorted(os.listdir(image_folder))
mask_filenames = sorted(os.listdir(mask_folder))

# Data normalization function
def normalize_data(data):
    return data / 255.0

# Initialize empty lists to store normalized images and masks
normalized_images = []
normalized_masks = []

# Iterate over the image and mask filenames
for i in range(len(image_filenames)):
    # Load image and mask
    image_filename = os.path.join(image_folder, image_filenames[i])
    mask_filename = os.path.join(mask_folder, mask_filenames[i])
    image = cv2.imread(image_filename)
    mask = cv2.imread(mask_filename, cv2.IMREAD_GRAYSCALE)
    new_size = (384, 96)
    image_resized = cv2.resize(image, new_size)
    mask_resized = cv2.resize(mask, new_size)
    # Normalize image and mask
    normalized_image = normalize_data(image_resized)
    normalized_mask = normalize_data(mask_resized)
    # Binarize the mask (optional)
    normalized_mask = np.where(normalized_mask > 0.5, 1, 0)

    # Append normalized images and masks to the respective lists
    normalized_images.append(normalized_image)
    normalized_masks.append(normalized_mask)

# Convert the image and mask lists to numpy arrays
normalized_images = np.array(normalized_images)
normalized_masks = np.array(normalized_masks)

# Choose the index of the image you want to test
image_index = 30

# Get the corresponding test image and mask
test_image = normalized_images[image_index]
test_mask = normalized_masks[image_index]

# Convert the test image and mask to numpy arrays
test_images = np.expand_dims(test_image, axis=0)
test_masks = np.expand_dims(test_mask, axis=0)

# Make predictions on the test image
predictions = model.predict(test_images)

# Display the test image, ground truth mask, and predicted mask
fig = plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(test_image)
plt.title("Test Image", fontsize=14)
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(test_mask, cmap='gray')
plt.title("Ground Truth Mask", fontsize=14)
plt.axis("off")

plt.subplot(1, 3, 3)
predicted_mask = np.squeeze(predictions)  # Remove the batch dimension
predicted_mask = np.where(predicted_mask > 0.6, 1, 0)  # Binarize the predicted mask
plt.imshow(predicted_mask, cmap='gray')
plt.title("Predicted Mask", fontsize=14)
plt.axis("off")

plt.tight_layout()
plt.show()

# Save the ground truth mask
plt.imsave("ground_truth_mask.png", test_mask, cmap='gray')

# Save the predicted mask
plt.imsave("predicted_mask.png", predicted_mask, cmap='gray')

# Flatten the ground truth mask and predicted mask
ground_truth_mask_flat = test_mask.flatten()
predicted_mask_flat = predicted_mask.flatten()

# Calculate precision, recall, and F1 score
precision, recall, _ = precision_recall_curve(ground_truth_mask_flat, predicted_mask_flat)
f1 = 2 * (precision * recall) / (precision + recall)

# Calculate ROC curve and AUC
fpr, tpr, _ = roc_curve(ground_truth_mask_flat, predicted_mask_flat)
roc_auc = auc(fpr, tpr)

# Calculate PSNR
def calculate_psnr(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    max_pixel = 1.0  # Assuming pixel values are in the range [0, 1]
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

# Calculate DRD
def calculate_drd(y_true, y_pred):
    drd = np.mean(np.abs(y_true - y_pred))
    return drd


# Calculate IoU and mAP
def calculate_iou(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    iou = np.sum(intersection) / np.sum(union)
    return iou



def calculate_ap(y_true, y_pred):
    # Flatten the masks
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()



psnr = calculate_psnr(test_mask, predicted_mask)
drd = calculate_drd(test_mask, predicted_mask)

# Calculate Intersection-over-Union (IoU) and mean Average Precision (mAP)
iou = calculate_iou(test_mask, predicted_mask)
ap = calculate_ap(test_mask, predicted_mask)

# Calculate Structural Similarity (SSIM)
ssim = structural_similarity(test_mask, predicted_mask)

# Print the evaluation metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("AUC:", roc_auc)
print("Peak Signal-to-Noise Ratio (PSNR):", psnr)
print("Dynamic Range Difference (DRD):", drd)
print("Intersection-over-Union (IoU):", iou)
print("Mean Average Precision (mAP):", ap)
print("Structural Similarity (SSIM):", ssim)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()



In [ ]:
pip install visualkeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.0/993.0 kB 56.5 MB/s eta 0:00:00
